In [1]:
import pandas as pd
from pymongo import MongoClient
import random

from mongo_aggregation_verbs import *

mongo_client = MongoClient('18.236.138.158', 27016)
database_reference = mongo_client.twitter

In [2]:
database_reference.collection_names()

collection_reference = database_reference.instructor_test_group

test_group = database_reference.instructor_test_group

In [3]:
match_empty_url_arrays = { MATCH : { "entities.urls" : [] } }

list(test_group.aggregate(
    [
        match_empty_url_arrays,
        { COUNT : "text" }
    ]
))

[{'text': 11121}]

In [4]:
job_hashtags = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
match_not_in_bad = { MATCH : { "text" : { "$in" : job_hashtags + location_hashtags } } }
project_to_text_keep_id = { PROJECT : { "text" : "$entities.hashtags.text" } }
project_to_id = { PROJECT : { "_id" : 1 } }

bad_ids = list(test_group.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_keep_id,
        unwind_text,
        project_to_lower,
        match_not_in_bad,
        project_to_id
    ]
))
bad_ids[:10], len(bad_ids)

([{'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')}],
 1835)

In [5]:
bad_ids = [bad_id['_id'] for bad_id in bad_ids]
bad_ids[:10]

[ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6df39136dd5f0001691533'),
 ObjectId('5a6df39136dd5f0001691533')]

In [6]:
not_in_bad_ids = { "$nin" : bad_ids }

In [8]:
not_in_bad_ids_and_no_url = { 
    "_id"           : not_in_bad_ids, 
    "entities.urls" : []
}

just_the_text = {
    "text" : 1,
    "_id"  : 0
}

In [9]:
test_group.find_one(
    not_in_bad_ids_and_no_url,
    just_the_text
)

{'text': "@SincerelyLegit Lol why at night? If it's boring I'll fall asleep"}

In [10]:
cur  = test_group.find(
    not_in_bad_ids_and_no_url,
    just_the_text
)

tweets = list(cur)
tweet_text = pd.DataFrame(tweets)

In [11]:
len(tweet_text)

11102

In [12]:
tweet_text.head()

,text
0,@SincerelyLegit Lol why at night? If it's bori...
1,@godtributes Lol
2,@KingBeyonceStan I need to binge on both now b...
3,@_QUEENSharnay Naw I’d i can’t put this skip o...
4,@sannicolaso I have things to do at home but y...


In [13]:
tweet_text.text = tweet_text.text.str.replace('http\S+|www.\S+', '', case=False)

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(tweet_text.text)
word_occurence = tfidf.transform(tweet_text.text).todense()

In [16]:
word_occurence.shape

(11102, 20582)

In [17]:
words = tfidf.get_feature_names()
word_sample = random.sample(words, 20)
word_occurence_m = pd.DataFrame(word_occurence, columns=words)
word_occurence_m[word_sample].head()

,tires,awesomeeeee,odds,clout,100brbr,total,holy,bluejays,newyork,thatkiddez,picklerinn,fortunate,witnessed,rl_miller,bob,carmella,danrather,westla,betsy,brinamoniquee
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from sklearn.decomposition import LatentDirichletAllocation

In [19]:
lda = LatentDirichletAllocation(n_topics=10, learning_method='batch')
lda.fit(word_occurence)

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [20]:
lda_df = pd.DataFrame(lda.components_, columns=words).T

In [21]:
def filter_topic(lda_df, index, threshold):
    return (lda_df[lda_df[index] > threshold][index]
            .sort_values(ascending=False))

In [22]:
filter_topic(lda_df, 0, 10)

realdonaldtrump    26.924098
life               14.433196
today              13.796060
like               12.909203
tonight            12.440967
hi                 11.932545
lol                11.809768
just               11.709171
thank              10.415115
lo                 10.157741
Name: 0, dtype: float64

In [23]:
filter_topic(lda_df, 1, 10)

good       17.543596
god        16.114882
morning    13.859482
just       13.609985
day        13.163371
thank      10.265703
Name: 1, dtype: float64

In [24]:
filter_topic(lda_df, 2, 10)

don     17.529542
mood    15.397384
wow     13.017690
time    12.512149
gt      12.051757
like    11.812582
just    11.767592
Name: 2, dtype: float64

In [30]:
filter_topic(lda_df, 3, 5)

good     11.035873
gonna     9.677585
la        9.457188
right     8.257726
just      7.964738
true      6.918964
need      6.860816
like      6.785791
ass       6.449890
heart     5.911852
tired     5.898605
music     5.106587
omg       5.063022
Name: 3, dtype: float64

In [26]:
filter_topic(lda_df, 4, 10)

love     17.597504
just     15.784500
like     11.736119
wait     11.312292
today    10.499123
Name: 4, dtype: float64

In [27]:
filter_topic(lda_df, 5, 10)

fuck      19.533498
just      19.283515
yes       16.218575
people    15.788082
know      15.775503
like      15.591380
love      13.543071
don       12.154129
bitch     11.227335
wanna     10.108777
lol       10.100001
Name: 5, dtype: float64

In [28]:
filter_topic(lda_df, 6, 6)

love       24.931024
good       11.150954
angeles     9.569691
sleep       9.182882
omg         9.061345
los         8.237702
best        7.957185
going       7.851733
really      7.655026
like        7.567627
lol         7.551425
couldn      7.519786
care        7.317565
just        7.006165
fall        6.931113
smh         6.906545
today       6.660393
time        6.586684
getting     6.421132
day         6.249168
Name: 6, dtype: float64

In [29]:
filter_topic(lda_df, 7, 7)

like      20.469175
lmao      14.012189
got       13.858762
really    12.019327
just      11.643017
great     11.011661
want      10.680874
need       9.494232
money      9.130007
make       8.750701
didn       8.128451
lol        7.969240
love       7.931216
crazy      7.606814
say        7.390890
know       7.127195
Name: 7, dtype: float64

In [ ]:
filter_topic(lda_df, 8, 7)

In [ ]:
filter_topic(lda_df, 9, 7)